In [1]:
from facet_ml.classification import cnn

from facet_ml.classification import cnn
import torch
from torch import nn
import numpy as np
from torchvision.models import resnet152
from pathlib import Path


# train on the GPU or on the CPU, if a GPU is not available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

# # Load data
# data_folder = (
#     Path(__file__).parent.parent.parent.parent / "ProcessedData" / "Coco_v5"
# )
data_folder = r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\ProcessedData\Coco_v5"
csv_path = r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\ProcessedData\Training_Data_20240216\2024_02_27_Rachel-C_Processed.csv"
h5_path = r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\ProcessedData\Training_Data_20240216\2024_02_16_Rachel-C_Training.h5"
dataloaders, datasizes = cnn.load_colloidal_datasets_coco(
    str(data_folder),
    num_workers=16,
    batch_size=2,
)


cuda


c:\Users\Jacob\miniconda3\envs\colloidal_crystal_env\lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [2]:
from facet_ml.classification import cnn
dir_path = Path(data_folder)
dataset = cnn.CocoColloidalDataset(str(dir_path / "train"),str(dir_path / "train" / "_annotations.coco.json"))

print(type(dataset[1][0]))
np.shape(dataset[1][0]), np.shape(dataset[1][1])

<class 'torch.Tensor'>


(torch.Size([3, 1024, 1280]), torch.Size([1024, 1280]))

In [3]:
print(type(dataloaders["train"]))
dl:torch.utils.data.DataLoader = dataloaders["train"]

np.shape( dl.dataset[0][0] )
image_inputs, target_inputs = next(iter(dl))

<class 'torch.utils.data.dataloader.DataLoader'>


In [4]:
image_inputs.shape,target_inputs.shape

(torch.Size([2, 3, 256, 256]), torch.Size([2, 1024, 1280]))

In [5]:

# Load model
model = cnn.get_model(2, resnet152(weights="DEFAULT"))
model = cnn.get_binary_model(2, resnet152(weights="DEFAULT"))
# model = cnn.UNet(3, 2)
model.to(device=device)

## Get model trianing information
criterion = nn.CrossEntropyLoss()

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0001)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

model, loss_dict, accuracy_dict = cnn.train_model(
    model,
    criterion,
    optimizer,
    lr_scheduler,
    num_epochs=50,
    dataloaders=dataloaders,
    dataset_sizes=datasizes,
)

# Make quick plot
import matplotlib.pyplot as plt

fig, axes = plt.subplots(ncols=2)
for ii, key in enumerate(loss_dict.keys()):
    # ax = axes[ii]
    x = np.array(range(len(loss_dict[key])))

    for ii, (label, dict_oi) in enumerate(
        [("Loss", loss_dict), ("Accuracy", accuracy_dict)]
    ):
        ax = axes[ii]
        y = np.array(dict_oi[key])

        ax.plot(x, y, label=f"{label}: {key}")
        ax.legend()

fig.savefig("Model_performance.png")
torch.save(model.state_dict(), "resnet152_trained.pth")


Epoch 0/49
----------


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from torchvision.models.segmentation import deeplabv3_resnet101

# Initialize DataLoader
dataloader = dl

# Load pre-trained DeepLabV3 model with ResNet-101 backbone
model = deeplabv3_resnet101(pretrained=True)
model.classifier[4] = nn.Conv2d(256, 1, kernel_size=1)  # Modify classifier for binary segmentation

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For binary segmentation
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in dataloader:
        images, targets = images.to(device), targets.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)['out']
        print(outputs.shape, targets.shape)  # Check shapes

        # Ensure targets are the same shape as outputs
        targets = targets.unsqueeze(1).float()  # Add channel dimension and convert to float

        # Compute loss
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

print('Training finished.')


c:\Users\Jacob\miniconda3\envs\colloidal_crystal_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Jacob\miniconda3\envs\colloidal_crystal_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([2, 1, 256, 256]) torch.Size([2, 1024, 1280])


ValueError: Target size (torch.Size([2, 1, 1024, 1280])) must be the same as input size (torch.Size([2, 1, 256, 256]))

In [ ]:
model(images)

NameError: name 'images' is not defined